In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import gensim
from gensim.phrases import Phrases, Phraser


ModuleNotFoundError: No module named 'gensim.phrases'

In [2]:
from os import listdir
from os.path import join
from gensim.utils import tokenize
import gzip
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import codecs


class GzippedCorpusStreamer(object):
    def __init__(self, corpus_fpath):
        self._corpus_fpath = corpus_fpath
        
    def __iter__(self):
        for line in gzip.open(self._corpus_fpath, "r", "utf-8"):
                yield list(tokenize(line,
                              lowercase=False,
                              deacc=False,
                              encoding='utf8',
                              errors='strict',
                              to_lower=False,
                              lower=False))

def learn_word_vectors(corpus_fpath, vectors_fpath):
    sentences = GzippedCorpusStreamer(corpus_fpath) 
    phrases = Phrases(sentences)
    bigram = Phraser(phrases)
    model = Word2Vec(bigram[sentences],
                     min_count=1,
                     size=100,
                     window=5, 
                     max_vocab_size=None,
                     workers=4,
                     sg=0,
                     iter=5)
    model.wv.save_word2vec_format(vectors_fpath, binary=False)


from time import time 

tic = time()
corpus_fpath = "/home/panchenko/tmp/gw.txt.gz"
vectors_fpath = "/home/panchenko/tmp/gw.w2v"
learn_word_vectors(corpus_fpath, vectors_fpath)
print(time()-tic)

tic = time()
corpus_fpath = "/home/panchenko/tmp/gigaword.txt.gz"
vectors_fpath = "/home/panchenko/tmp/gigaword-phraser.w2v"
learn_word_vectors(corpus_fpath, vectors_fpath)
print(time()-tic)

In [10]:
r2 = 
# for x in list(bigram[sentences]):
#     print(x)
#     break

In [12]:
for x in r2:
    print(x)

['rd_Rd', 'Michael_Stich', 'Germany', 'x_bt', 'Karim_Alami', 'Morocco', 'jpb', 'rw_reopens', 'BERLIN', 'rd_Rd', 'Anke_Huber', 'Germany', 'x_bt', 'Katarina', 'Maleeva_Bulgaria', 'Elena_Makarova', 'Russia', 'bt_Barbara', 'Rittner_Germany', 'x', 'Ann_Grossman', 'USA_bt', 'Gabriela_Sabatini', 'Argentina', 'x', 'Brenda_Schultz', 'Holland', 'bt_Silke', 'Meier_Germany', 'vog', 'rw', 'Tributes', 'pour', 'in', 'for', 'late', 'British', 'Labour_Party', 'leader', 'UNDATED', 'May_AFP', 'Tributes', 'poured', 'in', 'from', 'around', 'the', 'world', 'Thursday', 'to', 'the', 'late', 'Labour_Party', 'leader', 'John_Smith', 'who', 'died', 'earlier', 'from', 'a', 'massive', 'heart_attack', 'aged']
['In', 'Washington', 'the', 'US', 'State_Department', 'issued', 'a', 'statement', 'regretting', 'the', 'untimely_death', 'of', 'the', 'rapier', 'tongued', 'Scottish', 'barrister', 'and', 'parliamentarian']
['Mr', 'Smith', 'throughout', 'his', 'distinguished', 'career', 'in', 'government', 'and', 'in', 'oppositi

['He', 'said', 'he', 'was', 'not', 'aware', 'he', 'suffered', 'from', 'it', 'until', 'he', 'was', 'on', 'Death', 'Row']
['I_m', 'a', 'human', 'being', 'and', 'not', 'a', 'monster', 'said', 'Lawson', 'Tuesday', 'during', 'an_interview', 'with', 'NBC_television']
['It', 's', 'all', 'about', 'making', 'the', 'whole', 'story', 'public', 'in', 'order', 'to', 'help', 'somebody_else', 'who', 's', 'suffering', 'said', 'Marshall', 'Dayan', 'Lawson', 's', 'attorney']
['European', 'title', 'for', 'Romania', 'STOCKHOLM', 'May_AFP', 'Lavinia', 'Milosovici', 'Gina', 'Gogean', 'Nadia', 'Hategan', 'and', 'Simona', 'Amanar', 'won', 'the', 'women', 's', 'title', 'for', 'Romania', 'at', 'the', 'European', 'team', 'gymnastics', 'championships_here', 'on', 'Thursday']
['Result']
['Romania', 'pts']
['Russia']
['Ukraine']
['France']
['Belarus']
['Spain']
['Letvia']
['Israel']
['Greece']
['Italy', 'Violence', 'overshadows', 'Dominican_Republic', 'elections', 'by', 'Arturo_Gudino', 'SANTO_DOMINGO', 'May_AFP', 

['Authorities', 'here', 'have', 'only', 'permitted', 'US', 'firms', 'to', 'participate', 'in', 'the', 'local', 'insurance_market', 'since', 'in', 'response', 'to', 'strong', 'demands', 'by', 'Washington']
['Under', 'a', 'set', 'of', 'regulations', 'drafted', 'by', 'the', 'ministry', 'insurance_companies', 'from', 'other', 'countries', 'will', 'be_allowed', 'to', 'set_up', 'subsidiaries', 'here', 'on', 'a', 'reciprocal', 'basis', 'the', 'officials', 'said']
['Foreign', 'insurance_firms', 'can', 'file', 'applications', 'to', 'the', 'ministry', 'from', 'June', 'to', 'December', 'for', 'operating', 'licenses', 'they', 'said']
['Each', 'applicant', 'is', 'required', 'to', 'hold', 'a', 'minimum', 'paid', 'up', 'capital', 'of', 'two', 'billion', 'Taiwanese', 'dollars', 'million', 'US', 'they', 'said', 'adding_that', 'the', 'Department', 'of', 'Insurance', 'under', 'the', 'ministry', 'will', 'take', 'six_months', 'to', 'process', 'applications']
['The', 'move', 'is', 'in', 'line', 'with', 'lib

['Pro', 'government', 'forces', 'were', 'also', 'fighting', 'the', 'predominantly_Tutsi', 'RPF', 'on', 'Friday', 'at', 'Ruhengeri', 'in', 'the', 'northwest', 'of', 'the', 'country', 'and', 'at', 'Bugesera', 'in', 'the', 'south', 'Kabia', 'added']
['Rwanda_plunged', 'back', 'into', 'civil_war', 'between', 'the', 'RPF', 'drawn', 'mainly', 'from', 'the', 'Tutsi_minority', 'and', 'the', 'Hutu_dominated', 'army', 'and', 'pro', 'government', 'militias', 'after', 'Hutu_president', 'Juvenal_Habyarimana', 'was', 'killed', 'in', 'a', 'suspected_rocket', 'attack', 'on', 'his', 'plane', 'on', 'April']
['Aid_agencies', 'estimate', 'that', 'some', 'people', 'have_been', 'slain', 'in', 'wholesale', 'ethnic_bloodshed', 'during', 'the', 'weeks', 'that', 'followed', 'amid', 'a', 'massive', 'humanitarian', 'tragedy']
['Hundreds', 'of', 'thousands', 'of', 'others', 'have', 'fled_abroad', 'and', 'face', 'dire', 'conditions', 'in', 'camps', 'in', 'Tanzania_Burundi', 'and', 'other', 'neighbouring_countries',

['It', 'does_not', 'directly', 'involve', 'FIFA', 'because', 'it', 'is', 'a', 'friendly', 'tournament', 'afterall']
['If', 'they', 'contacted', 'us', 'we', 'would', 'simply', 'offer', 'them', 'this', 'advice', 'said', 'Blatter']
['Blatter', 'also', 'pleaded', 'for', 'a', 'little', 'understanding', 'of', 'Maradona', 's', 'cirumstances', 'which', 'he', 'said', 'made', 'him', 'more', 'likely', 'to', 'be', 'arrested', 'than', 'anybody_else']
['I', 'ask', 'you', 'this', 'if', 'he', 'were', 'not', 'a', 'footballer', 'would', 'there', 'have_been', 'a', 'problem', 'because', 'he', 'most_likely', 'would', 'not', 'have_been', 'caught']
['said', 'Blatter']
['Meanwhile', 'Blatter', 'told', 'a', 'press_conference', 'it', 'was', 'likely', 'that', 'FIFA', 's', 'executive_committee', 'meeting', 'next', 'Friday', 'would', 'agree', 'to', 'raise', 'the', 'World_Cup', 'format', 'to', 'teams', 'in', 'time', 'for', 'the', 'competition', 'an', 'increase', 'of', 'eight', 'sides']
['But', 'Blatter', 'said', 't

...

['Tony_Rominger', 'Switz', 'h']
[]
['Mikel_Zarrabeitia', 'Spain', 'at']
[]
['Pedro_Delgado', 'Spain']
[]
['Alex_Zulle', 'Switz']
['Oliverio', 'Rincon_Colombia']
['Luc_Leblanc', 'France']
['Vicente', 'Aparicio_Spain']
['Luis', 'Perez', 'Spain']
['Fernando_Escartin', 'Spain']
['Alberto', 'Camargo_Colombia', 'dt_mo', 'New', 'united', 'approach', 'on', 'Bosnia', 'peace', 'attacked', 'by', 'warring_parties', 'by', 'Michael_Anders', 'GENEVA', 'May_AFP', 'The', 'US', 'western_Europe', 'and', 'Russia', 'have', 'finally', 'agreed', 'on', 'a', 'joint', 'approach', 'to', 'put', 'Bosnian', 'peace', 'efforts', 'back', 'on', 'track', 'but', 'failed', 'Saturday', 'to', 'win', 'anything', 'other', 'than', 'criticism', 'from', 'the', 'warring_parties']
['The', 'peace_plan', 'endorsed', 'by', 'the', 'US', 'and', 'Russia', 'and', 'five', 'west', 'European', 'states', 'in', 'Geneva', 'late', 'Friday', 'would', 'provide', 'percent', 'of', 'Bosnia_Hercegovina', 'to', 'a', 'Croat_Moslem', 'federation', 'and'

['St_James', 'was', 'celebrating', 'a', 'starting', 'spot', 'nine', 'places', 'better_than', 'any', 'woman', 'at', 'Indy']
['It_feels', 'geat', 'to', 'be', 'where', 'I_m', 'at', 'she', 'said']
['From', 'here', 'anything', 's', 'possible']
['Penske', 'racer', 'Paul_Tracy', 'injured', 'in', 'a', 'practice', 'crash', 'Friday', 'was', 'released', 'from', 'Methodist_Hospital', 'but', 'not', 'cleared', 'to', 'drive', 'Saturday', 'ending', 'his', 'pole', 'bid']
['js_bb', 'Social_Democrat', 'joins', 'Brazilian', 'presidential_race', 'CONTAGEM', 'Brazil', 'May_AFP', 'The', 'center_left', 'Party', 'of', 'Brazilian_Social', 'Democracy', 'on', 'Saturday', 'named', 'Senator', 'Fernando_Henrique', 'Cardoso', 'its', 'candidate', 'for', 'the', 'October', 'presidential_elections']
['Cardoso', 'a', 'former', 'treasury_secretary', 'is', 'the', 'closest_rival', 'to', 'frontrunner', 'Luis_Inacio', 'Lula_da', 'Silva', 'of', 'the', 'leftist_Workers', 'Party', 'according', 'to', 'a', 'recent', 'poll']
['Seven

['There', 'has_been', 'no', 'official', 'reponse', 'to', 'his', 'kidnapping', 'and', 'murder', 'but', 'press_reports', 'said', 'he', 'had', 'succeeded', 'Wednesday', 'in', 'gaining', 'the', 'acquittal', 'of', 'an', 'Islamic_fundamentalist', 'being', 'tried', 'by', 'one', 'of', 'the', 'country', 's', 'special', 'anti_terrorism', 'courts']
['Japan', 'deports', 'record_high', 'number', 'of', 'foreigners', 'TOKYO_May', 'AFP', 'The', 'Japanese', 'authorities', 'last_year', 'deported', 'a', 'record', 'people', 'according', 'to', 'an', 'official', 'report', 'released', 'Sunday']
['Deportations', 'for', 'violating', 'immigration_laws', 'rose_percent', 'from']
['It', 'was', 'the', 'first_time', 'the', 'number', 'has', 'topped', 'said', 'the', 'report', 'by', 'the', 'justice_ministry', 'which', 'controls', 'immigration', 'offices']
['Illegal', 'workers', 'accounted_for', 'of', 'the', 'total', 'rising', 'percent', 'from', 'the', 'previous', 'year', 'it', 'said']
['Thais', 'topped', 'the', 'list',

['Almost', 'one', 'million', 'pilgrims', 'from', 'across', 'the', 'world', 'have', 'already', 'arrived', 'in', 'Mecca', 'for', 'the', 'pilgrimage', 'the', 'highpoint', 'of', 'which', 'will', 'be', 'on', 'Friday', 'when', 'they', 'climb', 'Mount', 'Arafat', 'the', 'official', 'Saudi', 'agency_SPA', 'said', 'Sunday']
['Last_year', 'more_than', 'two', 'million', 'people', 'took_part', 'in', 'the', 'pilgrimage']
['reopens_MONACO', 'Shortly_before', 'the', 'start', 'drivers', 'had', 'paid_homage', 'to', 'Senna', 'the', 'triple_world', 'champion', 'whose', 'death', 'has', 'tended', 'to', 'overshadow', 'that', 'of', 'rookie', 'Ratzenberger']
['Gathered', 'together', 'at', 'the', 'front', 'of', 'the', 'grid', 'where', 'the', 'first', 'two', 'places', 'had_been', 'left', 'clear', 'out', 'of', 'respect', 'to', 'the', 'dead', 'drivers', 'they', 'made', 'way', 'for', 'Brazilians', 'Christian_Fittipaldi', 'and', 'Rubens', 'Barichello', 'along_with', 'Ratzenberger', 's', 'German', 'teammate', 'Heinz

KeyboardInterrupt: 